# 구내식당 식수 인원 예측 AI 경진대회

### 1. 데이터 불러오기

In [61]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestRegressor

In [38]:
path = "/content/drive/MyDrive/Dacon/튜토리얼/data/구내식당_식사인원_예측_data"
os.chdir(path)

In [39]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submission = pd.read_csv("sample_submission.csv")

### 2. 데이터 살펴보기

데이터 설명

 - 일자
 - 요일
 - 본사정원수
 - 본사휴가자수
 - 본사출장자수
 - 시간외근무명령서승인건수
 - 현본사소속재택근무자수
 - 조식메뉴
 - 중식메뉴
 - 석식메뉴
 - **중식계  (타깃)**
 - **석식계 (타깃)**

In [40]:
train.head(3)

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0


In [41]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205 entries, 0 to 1204
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   일자              1205 non-null   object 
 1   요일              1205 non-null   object 
 2   본사정원수           1205 non-null   int64  
 3   본사휴가자수          1205 non-null   int64  
 4   본사출장자수          1205 non-null   int64  
 5   본사시간외근무명령서승인건수  1205 non-null   int64  
 6   현본사소속재택근무자수     1205 non-null   float64
 7   조식메뉴            1205 non-null   object 
 8   중식메뉴            1205 non-null   object 
 9   석식메뉴            1205 non-null   object 
 10  중식계             1205 non-null   float64
 11  석식계             1205 non-null   float64
dtypes: float64(3), int64(4), object(5)
memory usage: 113.1+ KB


In [42]:
submission.head()

,일자,중식계,석식계
0,2021-01-27,0,0
1,2021-01-28,0,0
2,2021-01-29,0,0
3,2021-02-01,0,0
4,2021-02-02,0,0


In [43]:
train['식사가능인원'] = train['본사정원수'] - train['본사휴가자수'] - train['현본사소속재택근무자수'] - train['본사출장자수']
test['식사가능인원'] = test['본사정원수'] - test['본사휴가자수'] - test['현본사소속재택근무자수'] - test['본사출장자수']

In [44]:
drops = ['본사정원수', '본사휴가자수', '현본사소속재택근무자수', '본사출장자수']
train.drop(drops, axis=1, inplace=True)
test.drop(drops, axis=1, inplace=True)

In [58]:
train['요일'] = train['요일'].map({'월':1, '화':2, '수':3, '목':4, '금':5})
test['요일'] = test['요일'].map({'월':1, '화':2, '수':3, '목':4, '금':5})

In [59]:
train.head()

,일자,요일,본사시간외근무명령서승인건수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,식사가능인원
0,2016-02-01,1,238,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0,2401.0
1,2016-02-02,2,319,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0,2378.0
2,2016-02-03,3,111,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0,2365.0
3,2016-02-04,4,355,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0,2277.0
4,2016-02-05,5,34,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0,2142.0


In [60]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205 entries, 0 to 1204
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   일자              1205 non-null   object 
 1   요일              1205 non-null   int64  
 2   본사시간외근무명령서승인건수  1205 non-null   int64  
 3   조식메뉴            1205 non-null   object 
 4   중식메뉴            1205 non-null   object 
 5   석식메뉴            1205 non-null   object 
 6   중식계             1205 non-null   float64
 7   석식계             1205 non-null   float64
 8   식사가능인원          1205 non-null   float64
dtypes: float64(3), int64(2), object(4)
memory usage: 84.9+ KB


In [62]:
train['일자'] = pd.to_datetime(train['일자'])
test['일자'] = pd.to_datetime(test['일자'])

In [72]:
train['month'] = train['일자'].dt.month
test['month'] = test['일자'].dt.month

train['day'] = train['일자'].dt.day
test['day'] = test['일자'].dt.day

In [74]:
drops2 = ['일자', '조식메뉴', '중식메뉴', '석식메뉴']
train.drop(drops2, axis=1, inplace=True)
test.drop(drops2, axis=1, inplace=True)

In [76]:
target = ['중식계', '석식계']

y_train1 = train[target[0]]
y_train2 = train[target[1]]

train.drop(target, axis=1, inplace=True)

In [78]:
model1 = RandomForestRegressor()
model2 = RandomForestRegressor()

model1.fit(train, y_train1)
model2.fit(train, y_train2)

RandomForestRegressor()

In [81]:
pred1 = model1.predict(test)
pred2 = model2.predict(test)

In [83]:
submission['중식계'] = pred1
submission['석식계'] = pred2

In [84]:
submission.to_csv('RandomForest.csv', index=False)